# Suspicious Windows Mangement Instrumentation DLL Loaded Via Microsoft Word
Detects DLL's Loaded Via Word Containing VBA Macros Executing WMI Commands

## Rule Content
```
- title: Suspicious Windows Mangement Instrumentation DLL Loaded Via Microsoft Word
  id: a457f232-7df9-491d-898f-b5aabd2cbe2f
  status: experimental
  description: Detects DLL's Loaded Via Word Containing VBA Macros Executing WMI Commands
  references:
  - https://medium.com/threatpunter/detecting-adversary-tradecraft-with-image-load-event-logging-and-eql-8de93338c16
  - https://www.carbonblack.com/2019/04/24/cb-tau-threat-intelligence-notification-emotet-utilizing-wmi-to-launch-powershell-encoded-code/
  - https://media.cert.europa.eu/static/SecurityAdvisories/2019/CERT-EU-SA2019-021.pdf
  author: Michael R. (@nahamike01)
  date: 2019/12/26
  tags:
  - attack.execution
  - attack.t1047
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 7
      Image:
      - '*\winword.exe'
      - '*\powerpnt.exe'
      - '*\excel.exe'
      - '*\outlook.exe'
      ImageLoaded:
      - '*\wmiutils.dll'
      - '*\wbemcomn.dll'
      - '*\wbemprox.dll'
      - '*\wbemdisp.dll'
      - '*\wbemsvc.dll'
    condition: selection
  falsepositives:
  - Possible. Requires further testing.
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"7" AND process_path.keyword:(*\\winword.exe OR *\\powerpnt.exe OR *\\excel.exe OR *\\outlook.exe) AND module_loaded.keyword:(*\\wmiutils.dll OR *\\wbemcomn.dll OR *\\wbemprox.dll OR *\\wbemdisp.dll OR *\\wbemsvc.dll))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()